## Avaliação por meio de um método de aprendizado de máquina

Os embeddings podem oferecer uma informação de proximidade de conceitos que o uso de Bag of Words não seria capaz. Mesmo assim, cada representação e preprocessamento tem sua vantagem e desvantagem e não existe um método que será sempre o melhor. Assim, para sabermos qual representação é melhor para uma tarefa, é importante avaliarmos em quais delas são maiores para a tarefa em questão. Como o foco desta prática não é a avaliação, iremos apenas apresentar o resultado, caso queira, você pode [assistir a video aula](https://www.youtube.com/watch?v=Ag06UuWTsr4&list=PLwIaU1DGYV6tUx10fCTw5aPnqypbbK_GJ&index=12) e [fazer a prática sobre avaliação](https://github.com/daniel-hasan/ap-de-maquina-cefetmg-avaliacao/archive/master.zip). Nesta parte, iremos apenas usar a avaliação para verificar qual método é melhor.  

Para que esta seção seja auto contida, iremos fazer toda a preparação que fizemos nas seções anteriores

**Criação da lista de stopwords e de vocabulário:**

In [1]:
# reset para liberar memória
%reset -f
# fim do reset para liberar memória

from embeddings.utils import get_embedding, KDTreeEmbedding
from sklearn.decomposition import PCA

emotion_words = {
                    "Sexism":{"mulher","homem","cis","trans","feminista","burra","feia","gorda","puta","menina"},
                    "Body":{"gorda", "gordo", "magro", "magra","magrela","magrelo","feio","feia","burra","feminista","mulher"},
                    "Racism":{"negro","negra", "preto", "preta", "racismo", "branco","branca","latino","inverso"},
                    "Ideology":{"esquerda","direita","esquerdopata","esquerdomacho","direitista","minion"},
                    "Homophobia":{"fufa","sapatão","gay","viado","hetero","bicha"},
                    "Origin":{"latino", "brasileiro", "europeu", "gringo"},
                    "Religion":{"muçulmano", "deus", "allah", "crente","cristão","evangélico"},
                    "Health":{"deficiente", "amputado", "amputa","cadeirante"},
                    "OtherLifestyle":{"vegano","vegetariano","hippie","rock","emo","nerd"},
                    "Migrants":{"gringo", "estrangeiro", "Brasil", "Portugal", "Angola"},
                    "Ageing":{"velho","velha","incapaz","senil","idoso","idosa"}
            }
dict_embedding = get_embedding("glove.pt.100.txt")

kdtree_embedding = KDTreeEmbedding(dict_embedding, "kdt_pt.p")

#obtem as stopwords
stop_words = set()
with open("datasets/stopwords.txt") as stop_file:
    stop_words = set(stop_word[:-1] for stop_word in stop_file)


#palavras chaves a serem consideradas
set_vocabulary = set()
for key_word, arr_related_words in emotion_words.items():
    set_vocabulary.add(key_word)
    set_vocabulary = set_vocabulary | set(arr_related_words)

#kdtree - para gerar o conjunto com palavras chaves e suas similares
vocabulary_expanded = []
for word in set_vocabulary:
    _, words = kdtree_embedding.get_most_similar_embedding(word,60)
    vocabulary_expanded.extend(words)
vocabulary_expanded = set(vocabulary_expanded)

10000: distribuída
20000: diferenciados
30000: socialite
40000: bárbaras
50000: seguro-desemprego
60000: interligada
70000: landi
80000: hurts
90000: jackeline
100000: cataluña
110000: héber
120000: calama
130000: afogue
140000: natalícios
150000: amostrada
160000: portageiros
170000: ozias
180000: banerjee
190000: crackdown
200000: kirchspielslandgemeinde
210000: yello
220000: picrodendraceae
230000: rochlitz
240000: illis
250000: oitis
260000: kalki
270000: autorizagäo
280000: goleminov
290000: mamita
300000: interessarmos
310000: cprp
320000: samitier
330000: dimitre
340000: montegranaro
350000: sanguineti
360000: wurmser
370000: villaronga
380000: zimbra
390000: salvini-plawen
400000: pankisi
410000: hi-c
420000: boggio
430000: super-pena
440000: imecc
450000: adamascados
Palavras ignoradas: 3




**Representações usadas**:Iremos avaliar a filtragem de stopwords e usando um vocabulário restrito da representação bag of words e também da representação usando a média de embeddings.

In [7]:
from embeddings.textual_representation import BagOfWords, AggregateEmbeddings,InstanceWisePreprocess

#gera as representações
aggregate = AggregateEmbeddings(dict_embedding, "avg")
embedding = InstanceWisePreprocess("embbeding",aggregate)

aggregate_stop = AggregateEmbeddings(dict_embedding, "avg",words_to_filter=stop_words)
emb_nostop = InstanceWisePreprocess("emb_nostop",aggregate_stop)


aggregate_keywords_exp = AggregateEmbeddings(dict_embedding, "avg",words_to_consider=vocabulary_expanded)
emb_keywords_exp = InstanceWisePreprocess("emb_keywords_exp",aggregate_keywords_exp)

bow_keywords = BagOfWords("bow_keywords_exp", words_to_consider=vocabulary_expanded)
bow = BagOfWords("bow", stop_words=stop_words)

arr_representations = [bow]

In [8]:
import pandas as pd
df_hate_speech = pd.read_csv("2021-07-20_portuguese_hate_speech_hierarchical_classification.csv",delimiter=";")

Abaixo, é executado um método de aprendizado  para cada representação. Esse processo pode demorar um pouco pois é feito a procura do melhor parametro do algoritmo. Algumas otimizações que talvez, você precise fazer é no arquivo `embedding/avaliacao_embedding.py` alterar o parametro `n_jobs` no método `obtem_metodo` da classe `OtimizacaoObjetivoRandomForest`. Esse parametro é responsável por utiizar mais threads ao executar o Random Forests.  O valor pode ser levemente inferior a quantidades de núcleos que seu computador tem, caso ele tenha mais de 2, caso contrário, o ideal é colocarmos `n_jobs=1`. Caso queira visualizar resultados mais rapidamente, diminua o valor da variável `num_trials` e `num_folds` abaixo. Atenção que `num_folds` deve ser um valor maior que um.

In [9]:
import pandas as pd
import optuna
from embeddings.avaliacao_embedding import calcula_experimento_representacao, OtimizacaoObjetivoRandomForest

# Método de aprendizado de máquina a ser usado
dict_metodo = {"random_forest":{"classe_otimizacao":OtimizacaoObjetivoRandomForest,
                                "sampler":optuna.samplers.TPESampler(seed=1, n_startup_trials=10)},
              }
df_amazon_reviews = pd.read_csv("merge.csv",delimiter=";")

#executa experimento com a representacao determinada e o método
for metodo, param_metodo in dict_metodo.items():
    for representation in arr_representations:
        print(f"===== Representação: {representation.nome}")
        #col_classe = ["Sexism"]
        col_classe = ["Sexism","Racism","Homophobia","Body"]
        num_folds = 5
        num_folds_validacao = 3
        num_trials = 2

        for col in col_classe:
            nom_experimento = f"{col}_{metodo}_"+representation.nome
            experimento = calcula_experimento_representacao(nom_experimento,representation,df_amazon_reviews,
                                                col,num_folds,num_folds_validacao,num_trials,
                                                ClasseObjetivoOtimizacao=param_metodo['classe_otimizacao'],
                                                    sampler=param_metodo['sampler'])
            print(f"Representação: {representation.nome} {col} concluida")

===== Representação: bow


[I 2022-01-21 00:41:46,235] A new study created in RDB with name: Sexism_random_forest_bow_fold_0
[I 2022-01-21 00:41:51,690] Trial 0 finished with value: 0.8795503548320895 and parameters: {'min_samples_split': 9, 'max_features': 95, 'num_arvores': 30}. Best is trial 0 with value: 0.8795503548320895.
[I 2022-01-21 00:41:56,127] Trial 1 finished with value: 0.8828575123940281 and parameters: {'min_samples_split': 7, 'max_features': 75, 'num_arvores': 30}. Best is trial 1 with value: 0.8828575123940281.


0.8810480349344978


[I 2022-01-21 00:41:59,678] A new study created in RDB with name: Sexism_random_forest_bow_fold_1
[I 2022-01-21 00:42:06,478] Trial 0 finished with value: 0.8907007115892286 and parameters: {'min_samples_split': 5, 'max_features': 80, 'num_arvores': 35}. Best is trial 0 with value: 0.8907007115892286.
[I 2022-01-21 00:42:12,369] Trial 1 finished with value: 0.8861134996603363 and parameters: {'min_samples_split': 11, 'max_features': 80, 'num_arvores': 45}. Best is trial 0 with value: 0.8907007115892286.


0.8677772600186393


...

0.8942218080149116


[I 2022-01-21 00:42:31,518] A new study created in RDB with name: Sexism_random_forest_bow_fold_3
[I 2022-01-21 00:42:37,979] Trial 0 finished with value: 0.8843338735304315 and parameters: {'min_samples_split': 3, 'max_features': 75, 'num_arvores': 50}. Best is trial 0 with value: 0.8843338735304315.
[I 2022-01-21 00:42:44,059] Trial 1 finished with value: 0.8799373783349146 and parameters: {'min_samples_split': 21, 'max_features': 80, 'num_arvores': 45}. Best is trial 0 with value: 0.8843338735304315.


0.9023999374609131


[I 2022-01-21 00:42:48,864] A new study created in RDB with name: Sexism_random_forest_bow_fold_4
[I 2022-01-21 00:42:53,984] Trial 0 finished with value: 0.8776282879488054 and parameters: {'min_samples_split': 19, 'max_features': 100, 'num_arvores': 30}. Best is trial 0 with value: 0.8776282879488054.
[I 2022-01-21 00:42:59,845] Trial 1 finished with value: 0.8830895380943135 and parameters: {'min_samples_split': 1, 'max_features': 75, 'num_arvores': 50}. Best is trial 1 with value: 0.8830895380943135.


0.9076798488325011
Representação: bow Sexism concluida


Como a experimentação é uma tarefa custosa, todos os resultados são salvos na pasta "resultados" - inclusive os valores dos parametros na classe optuna (a prática de avaliação apresenta mais detalhes da biblioteca Optuna). A macro f1 é uma métrica relacionada a taxa de acerto (se necessário, [veja a explicação neste video - tópico 2 e 3)](https://www.youtube.com/watch?v=u7o7CSeXaNs&list=PLwIaU1DGYV6tUx10fCTw5aPnqypbbK_GJ&index=13). Analise os resultados abaixo: qual representação foi melhor? A restrição de vocabulário ou eliminação de stopwords auxiliou? 

In [10]:
import os
import pandas as pd
from base_am.avaliacao import Experimento

arr_resultado = []
for resultado_csv in os.listdir("resultados"):
    if resultado_csv.endswith("csv"):
        nom_experimento = resultado_csv.split(".")[0]
        
        #carrega resultados previamente realizados
        experimento = Experimento(nom_experimento,[])
        experimento.carrega_resultados_existentes()
        
        #adiciona experimento
        num_folds = len(experimento.resultados)
        dict_resultados = {"nom_experimento":nom_experimento, 
                            "macro-f1":sum([r.macro_f1 for r in experimento.resultados])/num_folds}
        #resultados por classe
        for classe in experimento.resultados[0].mat_confusao.keys():

            dict_resultados[f"f1-{classe}"] = sum([r.f1_por_classe[classe] for r in experimento.resultados])/num_folds
            dict_resultados[f"precision-{classe}"] = sum([r.precisao[classe] for r in experimento.resultados])/num_folds
            dict_resultados[f"recall-{classe}"] = sum([r.revocacao[classe] for r in experimento.resultados])/num_folds

        arr_resultado.append(dict_resultados)

resultado = pd.DataFrame.from_dict(arr_resultado)
resultado.sort_values(['macro-f1'], ascending = False, inplace = True)
resultado

[I 2022-01-21 00:43:08,503] Using an existing study with name 'random_forest_bow_fold_0' instead of creating a new one.
[I 2022-01-21 00:43:09,373] Using an existing study with name 'random_forest_bow_fold_1' instead of creating a new one.
[I 2022-01-21 00:43:10,218] Using an existing study with name 'random_forest_bow_fold_2' instead of creating a new one.
[I 2022-01-21 00:43:11,120] Using an existing study with name 'Sexism_random_forest_bow_fold_0' instead of creating a new one.
[I 2022-01-21 00:43:11,986] Using an existing study with name 'Sexism_random_forest_bow_fold_1' instead of creating a new one.
[I 2022-01-21 00:43:12,883] Using an existing study with name 'Sexism_random_forest_bow_fold_2' instead of creating a new one.
[I 2022-01-21 00:43:13,685] Using an existing study with name 'Sexism_random_forest_bow_fold_3' instead of creating a new one.
[I 2022-01-21 00:43:14,529] Using an existing study with name 'Sexism_random_forest_bow_fold_4' instead of creating a new one.
[I 20

,nom_experimento,macro-f1,f1-0,precision-0,recall-0,f1-1,precision-1,recall-1
1,Sexism_random_forest_bow,0.890625,0.893057,0.901367,0.886019,0.888194,0.880360,0.897488
0,random_forest_bow,0.792263,0.923971,0.887215,0.963920,0.660555,0.810754,0.557481
3,random_forest_emb_keywords_exp,0.703635,0.905296,0.847843,0.971156,0.501973,0.779643,0.370442
2,random_forest_embbeding,0.643717,0.892070,0.828357,0.966415,0.395363,0.695559,0.276213
4,random_forest_bow_keywords_exp,0.623100,0.868929,0.826350,0.916186,0.377271,0.499625,0.303617


### Discussão

Estudando o resultado acima, ordenado em forma decrescente pelo critério macro $F1$ que leva em consideração as predições corretas, os falsos negativos e os falsos positivos do classificador para calcular o quão bom ele é em classificar os individuos de determinados grupos (quanto mais próximo de 1 esse valor for, melhor o classificador)

Assim, verificamos que a representação bag of words é a que apresenta o maior macro $F1$, ou seja, a que mais analisa corretamente os sentimentos das revisões dos usuários. Porém, como dito anteriormente, essa representação é sujeita a uma limitação de generalização, visto que ela não permite calcular a distância entre palavras e consequentemente, a construir estruturas de sinônimos ou analogias por exemplo.

Por fim, random_forest_bow é seguida pela random forest embedding e random forest embedding que desconsidera stopwords. Considerei isso um aspecto interessante, pois, stopwords em tese são palavras sem grande contribuição semântica para a extração do sentimento de sentenças, mas no caso presente, elas dão um incremento significativo (aproximadamente 2%) no macro $F1$ do classificador

## Bibliografia

Bolukbasi, T., Chang, K. W., Zou, J., Saligrama, V., & Kalai, A. (2016). **[Man is to computer programmer as woman is to homemaker? Debiasing word embeddings](https://arxiv.org/abs/1607.06520)**. 

Hartmann, N., Fonseca, E., Shulby, C., Treviso, M., Rodrigues, J., & Aluisio, S. (2017). [**Portuguese word embeddings: Evaluating on word analogies and natural language tasks.**](https://arxiv.org/abs/1708.06025)


Pennington, J., Socher, R., & Manning, C. D. (2014, October).**[GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf)**. In EMNLP 2015 


Scherer, Klaus R. **[What are emotions? And how can they be measured?](https://journals.sagepub.com/doi/pdf/10.1177/0539018405058216)**. Social science information, v. 44, n. 4, p. 695-729, 2005.

Shen, D., Wang, G., Wang, W., Min, M. R., Su, Q., Zhang, Y., Carin, L. (2018). [Baseline needs more love: On simple word-embedding-based models and associated pooling mechanisms](https://arxiv.org/pdf/1805.09843.pdf).




<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licença Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />Este obra está licenciado com uma Licença <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Atribuição-CompartilhaIgual 4.0 Internacional</a>.